In [ ]:
import torch
import utils
from bokeh.io import show, output_notebook
output_notebook() 

Faisons un peu d’apprentissage automatique ! Dans cet exercice, tu pourras jouer avec une simple tâche de régression 1d, et tu pourras expérimenter le concept d’espaces de fonctions, de paramètres, d’ensembles de données, d’optimisation et de généralisation.

Une chose à noter est que nous n’utiliserons pas de réseau de neurones artificiel ! La raison est que les réseaux neuronaux artificiels sont connus pour être difficiles à interpréter et à comprendre. Pour explorer ces idées, nous allons plutôt utiliser un espace de fonction appelé une base linéaire, qui est assez simple pour aider à développer les intuitions.

### Partie 1: L’espace de fonctions

L’espace de fonctions avec lequel nous allons travailler aujourd’hui a huit paramètres. Le code ci-dessous créera une fonction à partir de ses paramètres, puis la tracera. Le format du tracé devrait avec un peu de chance être familier : l’intrant est un nombre unique, montré sur l’axe des x, et l’extrant est aussi un nombre unique, montré sur l’axe des y.

Essaie plusieurs paramètres et vois quelles différentes fonctions en résultent! Quelques éléments à essayer:

 - tous les paramètres sont des zéros
 - un paramètre est 1, le reste sont des zéros (essaie ceci pour chaque paramètre différent
 - tous les paramètres ne sont pas des zéros
 - un paramètre est grand, les autres sont petits


In [ ]:
### Change the parameters using the sliders to choose a function!
show(utils.interactive_plot())

### Partie 2: Les fonctions cibles

Maintenant que nous comprenons comment notre espace de fonctions fonctionne, essayons de l’utiliser pour trouver une fonction spécifique. Appelons la fonction que nous essayons de trouver notre « fonction cible ». Dans le code ci-dessous, il y a plusieurs fonctions cibles, qui varient en difficulté. Pour chacune, essaie de deviner quels sont les bons paramètres en ajustant ta réponse jusqu’à ce qu’elle corresponde ! Si tu n’arrives pas à la faire correspondre parfaitement, essaie approximativement, où les grosses tendances sont correctes, mais manquent peut-être certains des petits détails.

Ce processus devrait te donner une idée de comment fonctionne un algorithme d’apprentissage automatique : comme toi, un algorithme d’apprentissage automatique ajuste des paramètres jusqu’à ce qu’il trouve une bonne correspondance.

(En passant, ne t’inquiète pas si tu ne réussis pas les plus durs ! Essaie seulement puis passe au prochain.)

In [ ]:
from utils import easy_target_1, \
                  easy_target_2, \
                  easy_target_3, \
                  easy_target_4, \
                  medium_target_1, \
                  medium_target_2, \
                  medium_target_3, \
                  medium_target_4, \
                  hard_target_1, \
                  hard_target_2, \
                  hard_target_3, \
                  hard_target_4

### Change this to choose a task
target_function = hard_target_3

show(utils.interactive_plot(target_function=target_function))

### Partie 3: Les ensembles de données

L’exercice que nous venons de faire était en fait un peu trop facile comparé à ce que doivent faire les algorithmes d’apprentissage automatique. C’est parce qu’en général, les algorithmes d’apprentissage automatique n’auront pas accès à toute la fonction cible : seulement un ensemble de données limité. Dans cette section, essaie de trouver la fonction cible comme précédemment ; mais cette fois, fait le en regardant seulement l’ensemble de données.

En résolvant ces problèmes, pose-toi les questions suivantes :
 - Suis-je arrivé à placer l’ensemble de données ?
 - La fonction que j’ai trouvée était-elle la bonne fonction cible ? Étais-je proche ?

(Encore une fois, ne t’inquiète pas de tout résoudre parfaitement, expérimente seulement puis passe au prochain.)

In [ ]:
from utils import easy_target_small_dataset,  \
                  easy_target_medium_dataset, \
                  easy_target_large_dataset,  \
                  medium_target_small_dataset,  \
                  medium_target_medium_dataset, \
                  medium_target_large_dataset,  \
                  hard_target_small_dataset,  \
                  hard_target_medium_dataset, \
                  hard_target_large_dataset

task = easy_target_medium_dataset # change this line to choose a task
dataset = task.dataset
target_function = task.target_function

show(utils.interactive_plot(target_function=target_function, train_points=dataset))


### Partie 4: La descente de gradient

Maintenant que tu peux apprécier l’effort que ça prend pour rechercher manuellement un espace de fonctions en ajustant les paramètres, c’est le moment que vous attendiez tous… c’est le temps de faire que l’ordinateur le fasse!

L’algorithme de recherche que nous allons utiliser s’appelle une descente de gradient. Il fonctionne en prenant des petits pas dans l’espace de fonctions en ajustant à chaque fois les paramètres pour avoir une fonction un peu meilleure. Chacun de ces pas est pris dans la direction de la « meilleure amélioration locale » appelée **gradient**. Pour calculer le gradient, il faut utiliser le calculus, mais pour cet exercice, tu n’as pas besoin de t’en faire ; on peut utiliser une librairie Python appelée torch pour le faire automatiquement.

Une autre chose dont nous avons besoin est la définition de « meilleure fonction ». Toi et moi pouvons simplement regarder une image pour voir si une fonction correspond bien aux points de données, mais comment pouvons-nous exprimer cela mathématiquement ? L’intuition que nous voulons capturer est que notre fonction devrait être près des points de données, donc, pour toute paire [x, fonction_cible(x)] dans notre ensemble de données, ma_fonction(x) devrait être près de fonction_cible(x). Une façon de mesurer la distance entre deux nombres est de prendre leur différence et la mettre au carré. (Question à laquelle réfléchir : qu’arriverait-il si nous ne la mettions pas au carré ?)

Concrètement : nous voulons trouver les paramètres de la fonction dont la distance moyenne des points de données est la plus petite possible. Comme nous « gagnons » quand elle est petite, ça s’appelle la **perte**. Le gradient de la perte (avec respect pour les paramètres) nous dit comment changer les paramètres pour mieux augmenter la perte, donc nous voulons actualiser les paramètres dans la direction opposée du gradient.

Voyons si on peut le coder!

In [ ]:
## First, let's pick a random target.
##    (Important: this is a "secret" function! In a real task, we wouldn't have access to this.
##     Our learning algorithm isn't allowed to use it at all, or it would be cheating.)
target_function = utils.get_random_target()

## Next, let's get some examples of our function in action.
## This is what our algorithm is allowed to look at.
dataset = utils.get_examples(target_function, n_datapoints=50) ## once your implementation is done, try playing with this number!

## Now, let's initialize our parameters.
## Since we are going to be searching the space with gradient descent, it doesn't matter too much where we start!
## Feel free to play around with these values.
parameters = [0., 0., 0., 0., 0., 0., 0., 0.]
parameters = torch.nn.Parameter(torch.tensor(parameters)) ## this line just sets up some stuff in the background so that we can take the gradient later

## How long to run our gradient descent process for. The longer we run it, the harder we've searched.
TOTAL_STEPS = 2000
## How big of a step to take. (We multiply the gradient by the learning rate before taking a step.)
## Smaller steps take longer to learn, but if the stepsize is too big, gradient descent breaks and the whole thing goes haywire.
## (You'll learn why gradient descent needs to take tiny steps if you study calculus!)
LEARNING_RATE = .01

## Let's set up our plot so we can watch the learning happen.
plot, my_function_datasource, sliders = utils.live_plot(dataset, utils.function_from_parameters(parameters), target_function)
show(plot, notebook_handle=True)

## Start the main loop of training.
for step_i in range(TOTAL_STEPS):
    ## The current parameters define a function.
    my_function = utils.function_from_parameters(parameters)
    
    ## Update the plot to reflect our current guess.
    utils.update_live_plot(parameters, my_function_datasource, sliders)
    print(f"Step {step_i+1} / {TOTAL_STEPS} ({100*(step_i+1)/TOTAL_STEPS:3}%)", end='\r')
        
    ## Compute the loss on the dataset.
    losses_list = [(my_function(x) - y)**2. for x, y in dataset]
    average_loss = sum(losses_list)/len(losses_list)
    
    ## Compute the gradient from the loss using torch.autograd.
    ## The gradient is a list of 8 numbers, corresponding to each of our 8 parameters.
    ## Each number in the gradient list tells us how to change its corresponding parameter.
    gradient = torch.autograd.grad(average_loss, parameters)[0]
    
    ## Update the parameters using the gradient.
    ##    Note that there is a little trick here. parameters is a list of numbers, and
    ##    gradient is also a list of numbers. But since they are both wrapped in torch.tensor(),
    ##    they behave differently from normal Python lists. One cool thing we can do is "elementwise" math.
    ##    Multiplying a torch.tensor is the same as multiplying *each element* in the torch.tensor,
    ##    and subtracting two torch.tensor objects (of the same length) is equivalent to subtracting
    ##    each element of the second from its corresponding element in the first.
    parameters = parameters - LEARNING_RATE*gradient

Et voilà -- ton premier algorithme d’apprentissage automatique ! Avec un peu de chance, tu comprends maintenant ce que fait un algorithme, et pourquoi. Aussi, tu peux maintenant apprécier comment c’est génial que la descente de gradient puisse rechercher un gros espace de fonctions si rapidement!

Bien sûr, ce n’est que le début. La régression 1d est bien à visualiser, mais un peu ennuyante. Les choses deviennent très intéressantes quand on pense à des fonctions qui opèrent de façon complexe dans des espaces à haute-dimension. Par exemple, il y a une fonction qui schématise des positions sur un échiquier aux mouvements d’échec optimaux -- si nous pouvons apprendre celui-là, nous pouvons battre n’importe qui aux échecs ! Ou la fonction qui schématise d’images du visage d’une personne à l’image de ce visage avec un museau de chien dessus, comme SnapChat le fait avec ses filtres. Les possibilités sont vraiment infinies.

Surprenamment, pour apprendre toutes ces fonctions géniales (et plus), nous pouvons presque utiliser exactement le même algorithme que nous avons utilisé ici ! La différence principale est que nous passons du simple espace de fonctions à 8 paramètres que nous avons utilisé aujourd’hui à un espace de fonctions de réseaux neuronaux artificiels profonds, qui a des millions et même des milliards de paramètres. Bien sûr, nous ne pourrions jamais rechercher un espace comme ceux-là manuellement ! Mais en utilisant la descente de gradient, que nous avons apprise aujourd’hui, nous pouvons le faire.